# Filtrado de hilos en 4chan

En este cuaderno se procesan las publicaciones de 4chan, obtenidas a través de [web scrapping](./Scraper-4chan). Estas publicaciones pertenecen a múltiples [hilos que son considerados pertinenes]() para el objetivo de la investigación, ya sea por su idiosincrática nocividad en el uso del lenguaje, por la expresión misma de ideaciones/intenciones suicidas, o por contener sugerencias para llevar a cabo el suicidio.

In [53]:
import fnmatch
import pandas
import os
import re

A raíz de que se han consultado (y se consultarán) múltiples hilos, este script lee todas las publicaciones de los hilos y realiza múltiples operaciones sobre ellos.

A su vez, se crea una lista de nombres de archivos ya procesados, para excluirlos de la lectura cuando se trabaje en sesiones posteriores.

In [36]:
# Función para encontrar archivos

def encontrar_archivos(pattern : str, path : str, lista_ex : list, verbose : bool = False):

    archivos = []

    for root, dirs, files in os.walk(path):
        for index, name in enumerate(files):

          if verbose:
            print(f'Archivo #{index + 1}: {name}')
          if fnmatch.fnmatch(name, pattern) and name not in lista_ex:
              archivos.append(os.path.join(root, name))

    return archivos

In [27]:
archivos_procesados = [

    '',
    '',

]

In [39]:
rutas_archivos = []
rutas_base = [
    './datos/hilos_4chan',
]

for ruta in rutas_base:
  rutas_archivos.extend(
    encontrar_archivos(
        pattern = '*.csv',
        path = ruta,
        lista_ex = archivos_procesados,
        verbose=True
      )
  )

Archivo #1: 17913796 - comments & replies.csv


Los archivos contienen saltos de línea dentro de los campos, y las respuestas a un comentario raíz tienen el prefijo "(REPLY) >>", seguido del número del usuario. Esto se eliminará de las publicaciones.

A su vez, convertiremos todo a minúsuculas.

In [126]:
def pre_procesar_dataset(dataset : pandas.DataFrame):

  # Se eliminan los saltos de línea y los nombres de usuario en las publicaciones/respuestas
  dataset['comment/reply'] = dataset['comment/reply'].str.replace('\n', ' ')
  dataset['comment/reply'] = dataset['comment/reply'].str.replace('((\(REPLY\))?\s>>.{8})|>', '', regex=True)
  dataset['comment/reply'] = dataset['comment/reply'].str.lstrip()
  # Se transforma todo a minúsculas
  dataset['comment/reply'] = dataset['comment/reply'].str.lower()
  # Se retiran las columnas que no se usarán
  columnas = [
      'post_id',
      'subject',
      'name',
      'is_op?',
  ]
  dataset.drop(columns=columnas, inplace=True)
  # Se renombran las columnas -> Esta adecuación se hace para darles un formato similar a los dataframes de Twitter, para que en algún momento sean más sencillos
  # de procesar bajo el mismo conjunto de instrucciones
  dataset.rename(
      columns= {
          'date_time' : 'Date',
          'comment/reply' : 'Content',
          'url' : 'URL'
      }
    )


In [143]:
# Se leen los archivos .csv encontrados
datasets = []

for ruta in rutas_archivos:
  dataset = pandas.read_csv(ruta)
  pre_procesar_dataset(dataset)
  datasets.append(
      {
          "nombre" : os.path.basename(ruta),
          "datos" : dataset
      }
    )

<class 'pandas.core.frame.DataFrame'>


In [144]:
# Observemos un ejemplo
datasets[0]['datos'].head()

,date_time,comment/reply,url
0,"Jun-21-2023, 17:10:10",rainy days edition,http://boards.4chan.org/bant/thread/17913796/h...
1,"Jun-21-2023, 17:13:52",are you guys excited for hazbin to drop,http://boards.4chan.org/bant/thread/17913796/h...
2,"Jun-21-2023, 17:22:25",i do not watch american cartoons no more. they...,http://boards.4chan.org/bant/thread/17913796/h...
3,"Jun-21-2023, 17:25:21",helluva rock,http://boards.4chan.org/bant/thread/17913796/h...
4,"Jun-21-2023, 17:35:21","i'm curious, i know there's been a lot of buzz...",http://boards.4chan.org/bant/thread/17913796/h...


Una vez preparados los datos, podemos guardar una copia de ellos para su futuro análisis.

In [147]:
ruta_guardado = "./datos_limpios/4chan"
sufijo = "_prepared.csv"

ruta_partes = ruta_guardado.split("/")[1:] # Quitamos el punto

try:

  for in in range()
    os.mkdir(ruta_guardado)
except IOError:
  pass

for dict_ in datasets:
  nombre = dict_['nombre']
  dataset = dict_['datos']
  ruta_guardado_full = f"{ruta_guardado}/{nombre.removesuffix('.csv')}{sufijo}"
  dataset.to_csv(ruta_guardado_full)

Además, podemos leer una serie de palabras relevantes de un archivo, para después obtener únicamente publicaciones que las contienen.